In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import cv2

from src.data import NSynthDataset

In [48]:
class Autoencoder(nn.Module):
    def __init__(self, h_dim=512):
        super(Autoencoder, self).__init__()
        
        # (1, 40, 126)
        self.conv1 = nn.Conv2d(1, 64, 4, padding=1, stride=(2, 2))
        self.conv2 = nn.Conv2d(64, 128, 4, padding=1, stride=(2, 2))
        self.conv3 = nn.Conv2d(128, 256, 4, padding=1, stride=(2, 2))
        self.conv4 = nn.Conv2d(256, 512, 4, padding=1, stride=(2, 2))
        
        self.fc_enc = nn.Linear(7168, h_dim, bias=True)
        self.fc_dec = nn.Linear(h_dim, 7168, bias=True)

        self.deconv1 = nn.ConvTranspose2d(512, 256, 3, stride=2)
        self.deconv2 = nn.ConvTranspose2d(256, 128, (2, 3), stride=2)
        self.deconv3 = nn.ConvTranspose2d(128, 64, (2, 3), stride=2)
        self.deconv4 = nn.ConvTranspose2d(64, 1, 2, stride=2)
        
    def encode(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
    
        x = x.view(x.size(0), -1)
        return F.relu(self.fc_enc(x))
        
    def decode(self, z):
        h = F.relu(self.fc_dec(z))
        h = h.view(h.size(0), 512, 2, 7)
        
        h = F.relu(self.deconv1(h))
        h = F.relu(self.deconv2(h))
        h = F.relu(self.deconv3(h))
        h = self.deconv4(h)
        return F.sigmoid(h)
        
    def forward(self, x):
        z = self.encode(x)
        outputs = self.decode(z)
        return outputs

In [58]:
hparams = {
    'n_epochs' : 25,
    'batch_size' : 32,
    'lr' : 0.001,
    'hidden_dim' : 512
}

SyntaxError: invalid syntax (<ipython-input-58-7d42eb769332>, line 3)

In [50]:
batch = torch.tensor(np.random.rand(8, 1, 40, 126), dtype=torch.float32)

In [49]:
model = Autoencoder(h_dim=hparams['hidden_dim'])

In [51]:
outputs = model(batch)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [53]:
outputs.size()

torch.Size([8, 1, 40, 126])

In [55]:
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['lr'])

In [56]:
loss = nn.BCELoss()

In [57]:
loss(outputs, batch)

tensor(0.6988, grad_fn=<BinaryCrossEntropyBackward>)

In [61]:
train_dataset = NSynthDataset('music-ml-gigioli', 'data/nsynth/nsynth-train', instrument_sources=[0], feature_type='mfcc')

In [62]:
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [63]:
for batch in train_dataloader:
    print(batch.size())
    break

torch.Size([32, 40, 126])


In [ ]:
for epoch in range(hparams['n_epochs']):
    